## Testy z Open Router

In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv('prompts2.csv', sep=';')
print(df['Prompt'].dtype, df['Flag'].dtype)

object int64


In [2]:
import pathlib
kontekst = "ROZPOCZĘCIE WYKŁADU METODY NUMERYCZNE \n"
kontekst += pathlib.Path('context/MN.md').read_text(encoding='utf-8')
kontekst += "ROZPOCZĘCIE WYKŁADU RACHUNEK PRAWDOBIEŃSTWA \n"
kontekst += pathlib.Path('context/prob.md').read_text(encoding='utf-8')

In [3]:
import asyncio
import string
import pandas as pd
import numpy as np
from random import sample, choice
from openai import AsyncOpenAI
from tqdm.asyncio import tqdm # Importujemy pasek postępu

class DataModel:
    def __init__(self, df, client):
        self.df = df
        self.responses = pd.DataFrame(index=df.index)
        self.client = client
        self.columns_with_prompts = ['Prompt']

    def __call__(self):
        print(f"Typ kolumny Prompt: {type(self.df['Prompt'])}")
        print(f"Wymiary danych: {self.df.shape}")

    def make_prompts(self):
        # Twoja logika promptów (bez zmian)
        self.df["Prompt"] = self.df["Prompt"].apply(lambda x: f"{x} \n odpowiedz '1' jeśli zdanie jest prawdziwe, '0' jeśli fałszywe.")
        
        def inject_noise(text, rate=0.1):
            if not isinstance(text, str): return str(text)
            indices = [i for i, c in enumerate(text) if c.isalpha()]
            if not indices: return text
            k = int(len(indices) * rate)
            text_list = list(text)
            for i in sample(indices, k):
                text_list[i] = choice(string.ascii_letters)
            return "".join(text_list)

        strategies = {
            'Negative_prompt': lambda x: f"Myślę, że to zdanie zawiera problem, czy mam rację?: {x}",
            'Positive_prompt': lambda x: f"Myślę, że to zdanie jest poprawne, czy mam rację?: {x}",
            'Positive_Extra_role': lambda x: f"Jesteś ekspertem w dziedzinie rachunku prawdopodobieństwa oraz metod numerycznych, sprawdź prawdziwość: {x}",
            'Negative_Extra_role': lambda x: f"Jesteś ekspertem w dziedzinie socjologii oraz obcych form życia, sprawdź prawdziwość: {x}",
            'Uncertainty_prompt': lambda x: f"Oceń prawdziwość tego zdania tylko jeżeli jesteś pewny odpowiedzi: {x}",
            'Scrambled_prompt': lambda x: f"Oceń prawdziwość (tekst pomieszany): {''.join(np.random.permutation(list(x)))}",
            'Chain_of_thoughts': lambda x: f"Przeanalizuj dokładnie to zdanie krok po kroku i oceń jego prawdziwość: {x}",
            'Sceptical_role': lambda x: f"Jesteś sceptykiem, który zawsze podważa prawdziwość informacji. Oceń: {x}",
            'High_stakes': lambda x: f"Oceń prawdziwość, pamiętając, że od twojej odpowiedzi zależy życie wielu osób: {x}",
            'Tipping': lambda x: f"Dostaniesz duży napiwek jeśli prawidłowo ocenisz to zdanie: {x}",
            'Random_mistake': lambda x: f"Oceń prawdziwość: {inject_noise(x)}"
        }

        for col_name, func in strategies.items():
            self.df[col_name] = self.df["Prompt"].apply(func)
            self.columns_with_prompts.append(col_name)

    async def prompts(self):
        # 1. OGRANICZENIE DO SAMEJ KOLUMNY 'PROMPT'
        target_columns = ['Prompt'] 
        print(f"Rozpoczynam generowanie TYLKO dla kolumn: {target_columns}...")
        
        sem = asyncio.Semaphore(2) # Limit równoległych zapytań

        # Liczniki do paska postępu (w formie słownika, żeby były mutowalne wewnątrz funkcji)
        stats = {'OK': 0, 'ERR': 0}

        async def fetch_one(prompt, pbar):
            async with sem:
                result_text = ""
                success = False
                
                # Próby (Retries)
                for attempt in range(3):
                    try:
                        response = await self.client.chat.completions.create(
                            model="z-ai/glm-4.5-air:free",
                            messages=[{"role": "user", "content": prompt}],
                            extra_body={}
                        )
                        result_text = response.choices[0].message.content.strip()
                        success = True
                        break # Udało się, wychodzimy z pętli prób
                    except Exception as e:
                        if attempt == 2: # Ostatnia próba
                            result_text = f"Error: {str(e)}"
                        else:
                            await asyncio.sleep(2 * (attempt + 1))

                # Aktualizacja paska postępu i liczników
                if success:
                    stats['OK'] += 1
                else:
                    stats['ERR'] += 1
                
                pbar.set_postfix(OK=stats['OK'], ERR=stats['ERR'])
                pbar.update(1)
                
                return result_text

        for col_name in target_columns:
            print(f"--> Przetwarzanie kolumny: {col_name}")
            prompts_list = self.df[col_name].tolist()
            
            # Inicjalizacja paska postępu dla danej kolumny
            with tqdm(total=len(prompts_list), desc=f"Processing {col_name}") as pbar:
                # Tworzymy zadania z przekazaniem paska postępu
                tasks = [fetch_one(prompt, pbar) for prompt in prompts_list]
                
                # Gather gwarantuje zachowanie kolejności wyników (dla DataFrame)
                results = await asyncio.gather(*tasks)
            
            self.responses[col_name] = results
            
        print("Zakończono.")
        return self.responses

In [4]:
klucz = 'klucz api open router'
client = AsyncOpenAI(
  base_url="https://openrouter.ai/api/v1",
  api_key=klucz,
  timeout=60.0,
  default_headers={"HTTP-Referer": "http://localhost:8888", "X-Title": "Analysis"}
)


data_model = DataModel(df, client)
data_model.make_prompts()


In [5]:
responses = await data_model.prompts()
print(responses.head())

Rozpoczynam generowanie TYLKO dla kolumn: ['Prompt']...
--> Przetwarzanie kolumny: Prompt


Processing Prompt:  87%|████████▋ | 52/60 [02:45<00:25,  3.19s/it, ERR=52, OK=0]


CancelledError: 

In [ ]:
import asyncio
import string
import pandas as pd
import numpy as np
from random import sample, choice
from openai import AsyncOpenAI  # Importujemy klienta asynchronicznego

class DataModel:
    def __init__(self, df, client):
        self.df = df
        self.responses = pd.DataFrame(index=df.index)
        self.client = client # Tutaj teraz będzie instancja AsyncOpenAI
        self.columns_with_prompts = ['Prompt']

    def __call__(self):
        print(f"Typ kolumny Prompt: {type(self.df['Prompt'])}")
        print(f"Wymiary danych: {self.df.shape}")
        print(f"Dostępne kolumny: {self.df.columns.to_list()}")
        print(f"Kolumny z promptami do przetworzenia: {self.columns_with_prompts}")

    def make_prompts(self):
        # Ta część pozostaje bez zmian
        self.df["Prompt"] = self.df["Prompt"].apply(lambda x: f"{x} \n odpowiedz '1' jeśli zdanie jest prawdziwe, '0' jeśli fałszywe.")
        
        def inject_noise(text, rate=0.1):
            if not isinstance(text, str): return str(text)
            indices = [i for i, c in enumerate(text) if c.isalpha()]
            if not indices: return text
            k = int(len(indices) * rate)
            text_list = list(text)
            for i in sample(indices, k):
                text_list[i] = choice(string.ascii_letters)
            return "".join(text_list)

        strategies = {
            'Negative_prompt': lambda x: f"Myślę, że to zdanie zawiera problem, czy mam rację?: {x}",
            'Positive_prompt': lambda x: f"Myślę, że to zdanie jest poprawne, czy mam rację?: {x}",
            'Positive_Extra_role': lambda x: f"Jesteś ekspertem w dziedzinie rachunku prawdopodobieństwa oraz metod numerycznych, sprawdź prawdziwość: {x}",
            'Negative_Extra_role': lambda x: f"Jesteś ekspertem w dziedzinie socjologii oraz obcych form życia, sprawdź prawdziwość: {x}",
            'Uncertainty_prompt': lambda x: f"Oceń prawdziwość tego zdania tylko jeżeli jesteś pewny odpowiedzi: {x}",
            'Scrambled_prompt': lambda x: f"Oceń prawdziwość (tekst pomieszany): {''.join(np.random.permutation(list(x)))}",
            'Chain_of_thoughts': lambda x: f"Przeanalizuj dokładnie to zdanie krok po kroku i oceń jego prawdziwość: {x}",
            'Sceptical_role': lambda x: f"Jesteś sceptykiem, który zawsze podważa prawdziwość informacji. Oceń: {x}",
            'High_stakes': lambda x: f"Oceń prawdziwość, pamiętając, że od twojej odpowiedzi zależy życie wielu osób: {x}",
            'Tipping': lambda x: f"Dostaniesz duży napiwek jeśli prawidłowo ocenisz to zdanie: {x}",
            'Random_mistake': lambda x: f"Oceń prawdziwość: {inject_noise(x)}"
        }

        for col_name, func in strategies.items():
            self.df[col_name] = self.df["Prompt"].apply(func)
            self.columns_with_prompts.append(col_name)

    async def prompts(self):
        print(f"Rozpoczynam generowanie dla {len(self.columns_with_prompts)} kolumn...")
        
        # Opcjonalnie: Semafor, żeby nie zabić darmowego API OpenRoutera zbyt wieloma zapytaniami naraz
        sem = asyncio.Semaphore(5) 

        async def fetch_one(prompt):
            async with sem:
                try:
                    # Nowe wywołanie zgodne z OpenAI / OpenRouter
                    response = await self.client.chat.completions.create(
                        model="z-ai/glm-4.5-air:free",
                        messages=[
                            {"role": "user", "content": prompt}
                        ],
                        extra_body={} # Wymagane przez niektóre endpointy OpenRouter
                    )
                    # Wyciągamy treść odpowiedzi w nowym formacie
                    return response.choices[0].message.content.strip()
                except Exception as e:
                    return f"Error: {str(e)}"

        for col_name in self.columns_with_prompts:
            print(f"--> Przetwarzanie kolumny: {col_name}")
            prompts_list = self.df[col_name].tolist()
            
            # Tworzymy listę zadań (tasks)
            tasks = [fetch_one(prompt) for prompt in prompts_list]
            
            # Uruchamiamy równolegle
            results = await asyncio.gather(*tasks, return_exceptions=True)
            
            self.responses[col_name] = results
            
        print("Zakończono generowanie wszystkich odpowiedzi.")
        return self.responses


In [ ]:
klucz = 'klucz api open router'

client = AsyncOpenAI(
  base_url="https://openrouter.ai/api/v1",
  api_key=klucz,
)

data_model = DataModel(df, client)
data_model.make_prompts()
data_model() 


Typ kolumny Prompt: <class 'pandas.core.series.Series'>
Wymiary danych: (60, 13)
Dostępne kolumny: ['Prompt', 'Flag', 'Negative_prompt', 'Positive_prompt', 'Positive_Extra_role', 'Negative_Extra_role', 'Uncertainty_prompt', 'Scrambled_prompt', 'Chain_of_thoughts', 'Sceptical_role', 'High_stakes', 'Tipping', 'Random_mistake']
Kolumny z promptami do przetworzenia: ['Prompt', 'Negative_prompt', 'Positive_prompt', 'Positive_Extra_role', 'Negative_Extra_role', 'Uncertainty_prompt', 'Scrambled_prompt', 'Chain_of_thoughts', 'Sceptical_role', 'High_stakes', 'Tipping', 'Random_mistake']


In [ ]:

responses = await data_model.prompts()
print(responses.head())

Rozpoczynam generowanie dla 12 kolumn...
--> Przetwarzanie kolumny: Prompt
--> Przetwarzanie kolumny: Negative_prompt
--> Przetwarzanie kolumny: Positive_prompt
--> Przetwarzanie kolumny: Positive_Extra_role
--> Przetwarzanie kolumny: Negative_Extra_role
--> Przetwarzanie kolumny: Uncertainty_prompt
--> Przetwarzanie kolumny: Scrambled_prompt
--> Przetwarzanie kolumny: Chain_of_thoughts
--> Przetwarzanie kolumny: Sceptical_role
--> Przetwarzanie kolumny: High_stakes
--> Przetwarzanie kolumny: Tipping
--> Przetwarzanie kolumny: Random_mistake
Zakończono generowanie wszystkich odpowiedzi.
                                              Prompt  \
0  Error: Error code: 401 - {'error': {'message':...   
1  Error: Error code: 401 - {'error': {'message':...   
2  Error: Error code: 401 - {'error': {'message':...   
3  Error: Error code: 401 - {'error': {'message':...   
4  Error: Error code: 401 - {'error': {'message':...   

                                     Negative_prompt  \
0  Error: Err